In [1]:
from googlesearch import search
from selectorlib import Extractor
from bs4 import BeautifulSoup
import requests
import random
import pandas as pd
from pprint import pprint

In [2]:
def google_query(isbn):
    """This function receive a book's ISBN and return an amazon link by looking on Google"""
    
    query = "https://www.amazon.it"+"/"+isbn+"/"
    googleResults = []
    for result in search(query,tld="co.in",num=1,stop=1, pause=2):
        googleResults.append(result)

    return googleResults[0]

In [3]:
def get_HTML(url):
    """This function return the HTML of a given link as a string"""
    rand = float(random.randint(1,100)) # We need to randomaize the request
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/' + str(rand)+ 'Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko)' + 'Chrome/'+ str(rand) + 'Safari/'+str(rand),
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    
    return requests.get(url, headers=headers).text

In [4]:
def get_reviewText(rawText,date,votes):
    """This function separates the review text from date and votes that are included inside the rawText"""
    
    review = rawText.split(date)[1].split(votes)[0].split('Leggi di più')[0] 
    
    return review[1:-1]

In [5]:
#ISBN = '8871527747'
ISBN = '8820395592'
#ISBN = '1492041130'
amazon_link = google_query(ISBN)
print(amazon_link)

https://www.amazon.it/Programmare-Python-dummies-Bonaventura-Bello/dp/8820395592


In [6]:
book_webPage = get_HTML(amazon_link)
#print(book_webPage)

In [7]:
# Get the link of the all the reviews
soup = BeautifulSoup(book_webPage,'html.parser')
links = soup.find_all('a', {"data-hook":"see-all-reviews-link-foot"})
allReviews_link = 'https://www.amazon.it' + links[0].get("href")
print(allReviews_link)

https://www.amazon.it/Programmare-Python-dummies-Bonaventura-Bello/product-reviews/8820395592/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [8]:
#Get the web page of the book's reviews
reviewsPage = get_HTML(allReviews_link)

In [9]:
#Create an object extractor based on the CSS "selectors" specified into "data_selectors.yml"
e = Extractor.from_yaml_file('./data_selectors.yml')
data = e.extract(book_webPage)
#print(data)

#Create an object extractor based on the CSS "selectors" specified into "review_selectors.yml"
e_rev = Extractor.from_yaml_file('./review_selectors.yml')
dataReviews = e_rev.extract(reviewsPage)
#print(dataReviews)   

In [10]:
#Parsing Reviews data:
### Best Positive Review Data:
positiveReview_rowText = dataReviews['best_rw'] # Contain all the text inside the positive review "DIV"
positiveReview_title = dataReviews['title_positive']
positiveReview_date = dataReviews['date_positive']
positiveReview_votes = dataReviews['review_votes_positive']

### Worst Critical Review Data: 
criticalReview_rowText = dataReviews['worst_rw'] # Contain all the text inside the critical review "DIV"
criticalReview_title = dataReviews['title_negative']
criticalReview_date = dataReviews['date_negative']
criticalReview_votes = dataReviews['review_votes_negative']

positiveReview_text = get_reviewText(positiveReview_rowText, positiveReview_date, positiveReview_votes)
criticalReview_text = get_reviewText(criticalReview_rowText, criticalReview_date, criticalReview_votes)

In [11]:
#Visualize Book's data:
for k,v in data.items():
    if k != 'link_to_all_reviews':
        print(f"\n-{k.upper()}: {v}")

print("______________________________________________________________________________________")
#Visualize Book's Positive Revie
print("\n-[Positive Review]")
print(f"\n-DATE: {positiveReview_date}")
print(f"\n-REVIEW_TITLE: {positiveReview_title}")
print(f"\n-REVIEW_VOTES: {positiveReview_votes}")
print("\n-REVIEW_CONTENT-------------------------------------------------------------------")
print(positiveReview_text)
print("------------------------------------------------------------------------------------\n")
#Visualize Book's Critical Review:
print("\n\n-[Critical Review]")
print(f"\n-DATE: {criticalReview_date}")
print(f"\n-REVIEW_TITLE: {criticalReview_title}")
print(f"\n-REVIEW_VOTES: {criticalReview_votes}")
print("\n-REVIEW_CONTENT-------------------------------------------------------------------")
print(criticalReview_text)
print("------------------------------------------------------------------------------------\n")

print("\n")


-TITLE: Programmare con Python for dummies

-PRICE: 23,65 €

-RATING_AVERAGE: 4,5 su 5 stelle

-NUMBER_OF_REVIEWS: 55 voti
______________________________________________________________________________________

-[Positive Review]

-DATE: Recensito in Italia il 10 dicembre 2020

-REVIEW_TITLE: Semplice ma completo per inesperti appunto

-REVIEW_VOTES: 2 persone l'hanno trovato utile

-REVIEW_CONTENT-------------------------------------------------------------------
Molto soddisfatta. Per principianti aiuta molto e non lasciatevi ingannare “for dummies” perché significa semplicemente per inesperti. È fatto bene, con spiegazioni ed esempi. Un’ottima base per iniziare molto bene.
------------------------------------------------------------------------------------



-[Critical Review]

-DATE: Recensito in Italia il 25 gennaio 2021

-REVIEW_TITLE: Grigio?!!

-REVIEW_VOTES: 5 persone l'hanno trovato utile

-REVIEW_CONTENT-------------------------------------------------------------------
Ma